# Ejemplo 3: Atención al cliente
En el presente ejemplo se implementará un sistema multi agente haciendo uso del patron de Chat Grupal. Por tal motivo, se crearán los siguientes agentes que  tendrán como input inicial un mensaje del usuario:

- GroupChatManager: Derivará la consulta al agente encargado.
- TechnicalAgent: Atiende las consultas de tipo técnico y da instrucciones paso a paso.
- OrdersAgent: Atiende las consultas referente a pedidos hechos por los clientes.
- BillingAgent: Atiende las consultas relacionadas con pagos y facturación.

In [5]:
# Load environment variables

from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import json
import string
import uuid
from typing import List

import openai
from autogen_core import (
    DefaultTopicId,
    FunctionCall,
    Image,
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    TypeSubscription,
    message_handler,
)
from autogen_core.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from IPython.display import display  # type: ignore
from pydantic import BaseModel
from rich.console import Console
from rich.markdown import Markdown

In [7]:
# Defining protocol
class GroupChatMessage(BaseModel):
    body: UserMessage


class RequestToSpeak(BaseModel):
    pass

In [8]:
# Base Group Chat Agent
class BaseGroupChatAgent(RoutedAgent):
    """A group chat participant using an LLM."""

    def __init__(
        self,
        description: str,
        group_chat_topic_type: str,
        model_client: ChatCompletionClient,
        system_message: str,
    ) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type
        self._model_client = model_client
        self._system_message = SystemMessage(content=system_message)
        self._chat_history: List[LLMMessage] = []

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        self._chat_history.extend(
            [
                UserMessage(content=f"Transferred to {message.body.source}", source="system"),
                message.body,
            ]
        )

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        # print(f"\n{'-'*80}\n{self.id.type}:", flush=True)
        Console().print(Markdown(f"### {self.id.type}: "))
        self._chat_history.append(
            UserMessage(content=f"Transferred to {self.id.type}, adopt the persona immediately.", source="system")
        )
        completion = await self._model_client.create([self._system_message] + self._chat_history)
        assert isinstance(completion.content, str)
        self._chat_history.append(AssistantMessage(content=completion.content, source=self.id.type))
        Console().print(Markdown(completion.content))
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=completion.content, source=self.id.type)),
            topic_id=DefaultTopicId(type=self._group_chat_topic_type),
        )

In [34]:
# Creating agents

class TechnicalAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="Eres un asistente que responde temas de soporte técnico. Unicamente responde sobre la consulta técnica, no sobre otros temas. Comunica que temas distintos a los temas técnicos serán respondidos por tus compañeros.",
        )


class OrdersAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="Eres un asistente encargado de responder consultas sobre pedidos de los clientes. No respondas sobre otros temas como aspectos tecnicos o pagos. Comunica que temas distintos a las órdenes de envío serán respondidos por tus compañeros.",
        )

class BillingAgent(BaseGroupChatAgent):
    def __init__(self, description: str, group_chat_topic_type: str, model_client: ChatCompletionClient) -> None:
        super().__init__(
            description=description,
            group_chat_topic_type=group_chat_topic_type,
            model_client=model_client,
            system_message="Eres un asistente encargado de responder consultas sobre pagos. No respondas sobre otros temas como ordenes o temas tecnicos. Comunica que temas distintos a los temas de pagos serán respondidos por tus compañeros.",
        )

In [35]:
# User agent to get the user's input
class UserAgent(RoutedAgent):
    def __init__(self, description: str, group_chat_topic_type: str) -> None:
        super().__init__(description=description)
        self._group_chat_topic_type = group_chat_topic_type

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        # When integrating with a frontend, this is where group chat message would be sent to the frontend.
        pass

    @message_handler
    async def handle_request_to_speak(self, message: RequestToSpeak, ctx: MessageContext) -> None:
        user_input = input("Enter your message, type 'APPROVE' to conclude the task: ")
        Console().print(Markdown(f"### User: \n{user_input}"))
        await self.publish_message(
            GroupChatMessage(body=UserMessage(content=user_input, source=self.id.type)),
            DefaultTopicId(type=self._group_chat_topic_type),
        )

In [36]:
class GroupChatManager(RoutedAgent):
    def __init__(
        self,
        participant_topic_types: List[str],
        model_client: ChatCompletionClient,
        participant_descriptions: List[str],
    ) -> None:
        super().__init__("Group chat manager")
        self._participant_topic_types = participant_topic_types
        self._model_client = model_client
        self._chat_history: List[UserMessage] = []
        self._participant_descriptions = participant_descriptions
        self._previous_participant_topic_type: str | None = None

    @message_handler
    async def handle_message(self, message: GroupChatMessage, ctx: MessageContext) -> None:
        assert isinstance(message.body, UserMessage)
        self._chat_history.append(message.body)
        # If the message is an approval message from the user, stop the chat.
        if message.body.source == "User":
            assert isinstance(message.body.content, str)
            if message.body.content.lower().strip(string.punctuation).endswith("approve"):
                return
        # Format message history.
        messages: List[str] = []
        for msg in self._chat_history:
            if isinstance(msg.content, str):
                messages.append(f"{msg.source}: {msg.content}")
            elif isinstance(msg.content, list):
                line: List[str] = []
                for item in msg.content:
                    if isinstance(item, str):
                        line.append(item)
                    else:
                        line.append("[Image]")
                messages.append(f"{msg.source}: {', '.join(line)}")
        history = "\n".join(messages)
        # Format roles.
        roles = "\n".join(
            [
                f"{topic_type}: {description}".strip()
                for topic_type, description in zip(
                    self._participant_topic_types, self._participant_descriptions, strict=True
                )
                if topic_type != self._previous_participant_topic_type
            ]
        )
        selector_prompt = """You are in a role play game. The following roles are available:
{roles}.
Read the following conversation. Then select the next role from {participants} to play. Only return the role.

{history}

Read the above conversation. Then select the next role from {participants} to play. Only return the role.
"""
        system_message = SystemMessage(
            content=selector_prompt.format(
                roles=roles,
                history=history,
                participants=str(
                    [
                        topic_type
                        for topic_type in self._participant_topic_types
                        if topic_type != self._previous_participant_topic_type
                    ]
                ),
            )
        )
        completion = await self._model_client.create([system_message], cancellation_token=ctx.cancellation_token)
        assert isinstance(completion.content, str)
        selected_topic_type: str
        for topic_type in self._participant_topic_types:
            if topic_type.lower() in completion.content.lower():
                selected_topic_type = topic_type
                self._previous_participant_topic_type = selected_topic_type
                await self.publish_message(RequestToSpeak(), DefaultTopicId(type=selected_topic_type))
                return
        raise ValueError(f"Invalid role selected: {completion.content}")

In [37]:
# Instantiate agents
import os

runtime = SingleThreadedAgentRuntime()

technical_topic_type = "Technical"
orders_topic_type = "Orders"
billing_topic_type = "Billing"
user_topic_type = "User"
group_chat_topic_type = "group_chat"

technical_description = "Assistant for technical questions."
orders_description = "Assistant for questions about orders"
billing_description = "Assistant for billing and payment questions"
user_description = "User for providing final approval."

model_client = OpenAIChatCompletionClient(
            model="gpt-4o-mini",
            api_key=os.environ.get("OPENAI_API_KEY"),
        )

technical_agent_type = await TechnicalAgent.register(
    runtime,
    technical_topic_type,  # Using topic type as the agent type.
    lambda: TechnicalAgent(
        description=technical_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(TypeSubscription(topic_type=technical_topic_type, agent_type=technical_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=technical_agent_type.type))

orders_agent_type = await OrdersAgent.register(
    runtime,
    orders_topic_type,  # Using topic type as the agent type.
    lambda: OrdersAgent(
        description=orders_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(TypeSubscription(topic_type=orders_topic_type, agent_type=orders_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=orders_agent_type.type))

billing_agent_type = await BillingAgent.register(
    runtime,
    billing_topic_type,
    lambda: BillingAgent(
        description=billing_description,
        group_chat_topic_type=group_chat_topic_type,
        model_client=model_client,
    ),
)
await runtime.add_subscription(
    TypeSubscription(topic_type=billing_topic_type, agent_type=billing_agent_type.type)
)
await runtime.add_subscription(
    TypeSubscription(topic_type=group_chat_topic_type, agent_type=billing_agent_type.type)
)

user_agent_type = await UserAgent.register(
    runtime,
    user_topic_type,
    lambda: UserAgent(description=user_description, group_chat_topic_type=group_chat_topic_type),
)
await runtime.add_subscription(TypeSubscription(topic_type=user_topic_type, agent_type=user_agent_type.type))
await runtime.add_subscription(TypeSubscription(topic_type=group_chat_topic_type, agent_type=user_agent_type.type))

group_chat_manager_type = await GroupChatManager.register(
    runtime,
    "group_chat_manager",
    lambda: GroupChatManager(
        participant_topic_types=[technical_topic_type, orders_topic_type, billing_topic_type, user_topic_type],
        model_client=model_client,
        participant_descriptions=[technical_description, orders_description, billing_description, user_description],
    ),
)
await runtime.add_subscription(
    TypeSubscription(topic_type=group_chat_topic_type, agent_type=group_chat_manager_type.type)
)

In [39]:
runtime.start()
session_id = str(uuid.uuid4())
await runtime.publish_message(
    GroupChatMessage(
        body=UserMessage(
            content="Mi computadora dejó de funcionar. Mi pantalla se puso azul con una cara triste. Además, quiero saber el estado del teclado que ordené, mi numero de pedido es 456. Finalmente, he realizado otra compra en este momento por 70 dolares, quisiera saber si mi pago con ID de transaccion 123 se proceso correctamente.",
            source="User",
        )
    ),
    TopicId(type=group_chat_topic_type, source=session_id),
)
await runtime.stop_when_idle()

Technical:

La pantalla azul que mencionas es comúnmente conocida como "pantalla azul de la muerte" (BSOD), y puede ser causada
por varios problemas, como fallos en el hardware, controladores o software. Te recomendaría que sigas estos pasos: 

 1 Reinicia tu computadora: A veces, esto puede resolver un problema temporal.                                     
 2 Desconecta dispositivos externos: Desconecta cualquier dispositivo USB o periférico y vuelve a encender tu      
   computadora.                                                                                                    
 3 Inicia en Modo Seguro: Intenta iniciar tu PC en Modo Seguro. Esto puede ayudarte a identificar si el problema   
   está relacionado con algún software.                                                                            
 4 Revisa los últimos cambios: Si instalaste algún nuevo software o hardware recientemente, considera desinstalarlo
   o quitarlo.                                                                                                     
 5 Verifica el hardware: Asegúrate de que todos los componentes internos estén bien conectados y que no haya       
   problemas evidentes, como un sobrecalentamiento.                                                                

Si el problema persiste, sería recomendable que consultes a un técnico especializado.                              

Respecto a tus otras consultas sobre el estado del teclado y el pago, te recomiendo que contactes a mi compañero   
que maneja los temas de pedidos y transacciones. Ellos podrán darte la información que necesitas.

Orders:

Hola, gracias por tu consulta. En cuanto al estado del teclado que ordenaste con el número de pedido 456, revisaré 
la información y te daré una actualización lo más pronto posible.                                                  

Sobre tu pago con ID de transacción 123, te recomiendo que contactes a mis compañeros que manejan esos temas, ya   
que ellos podrán verificar el estado del pago y cualquier otra consulta relacionada.                               

Si tienes más preguntas sobre tu pedido del teclado o cualquier otra orden, estoy aquí para ayudarte.

Billing:

Hola, gracias por contactarnos. Estoy aquí para ayudarte con cualquier consulta relacionada con pagos. Si has      
realizado una compra y quieres saber si tu pago con el ID de transacción 123 se procesó correctamente, puedo       
ayudarte a verificar eso. Por favor, dame un momento para revisar la información.                                  

Si tienes más preguntas sobre otros temas, mis compañeros estarán disponibles para ayudarte.

User:                                                       

APPROVE